In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc # garbage collection module to release memory usage in time
import time
import warnings

os.chdir('/Users/jeremy_gp/Desktop/Project2/data')

warnings.filterwarnings('ignore')

%matplotlib inline

In [39]:
# stdt, nddt = 19570101, 20161231
stdt, nddt = 20010101, 20201231

# load firm characteristics data
data_ch = pd.read_csv('GKX_20201231.csv')
data_ch = data_ch[(data_ch['DATE']>=stdt)&(data_ch['DATE']<=nddt)].reset_index(drop=True)
data_ch['DATE'] = pd.to_datetime(data_ch['DATE'],format='%Y%m%d')+pd.offsets.MonthEnd(0)
characteristics = list(set(data_ch.columns).difference({'permno','DATE','SHROUT','mve0','sic2','RET','prc'}))

data_ch.head()

,permno,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia
0,10001,2001-01-31,24355.500,0.012821,9.8750,2498,0.037079,0.001375,0.281788,8.395576,...,0.020711,1.098587e-06,0.027778,0.017710,0.972710,0.426715,4.200000e+00,49.0,0.868139,-0.104925
1,10002,2001-01-31,78332.625,0.088435,10.0000,8526,0.206346,0.042579,0.050021,8.067022,...,0.033991,6.509871e-06,0.134328,0.054790,1.368962,0.759666,4.200000e+00,60.0,0.680296,-0.152630
2,10012,2001-01-31,39836.000,0.500000,3.0000,20897,2.470629,6.104008,-1.170178,11.360419,...,0.138777,9.482216e-08,0.129412,0.075671,0.465917,7.007556,8.756593e-09,36.0,0.061049,-0.397365
3,10016,2001-01-31,379569.500,0.030726,23.0625,16964,0.449866,0.202379,0.391222,12.024414,...,0.054578,5.643552e-08,0.070769,0.040708,1.242227,8.102766,1.833562e-08,38.0,0.287808,-0.227582
4,10019,2001-01-31,28945.000,0.071429,3.7500,8270,2.249729,5.061279,0.203106,9.294773,...,0.131620,3.206363e-07,0.435897,0.120324,0.983488,16.163956,7.497863e-09,38.0,0.552262,0.036872


In [40]:
wuxin_data = data_ch[['DATE','permno','RET']]
wuxin_data.to_csv('wuxin_data.csv',index=False)

In [41]:
data_ch_top = data_ch.sort_values('mvel1',ascending=False).groupby('DATE').head(1000).reset_index(drop=True)
data_ch_bot = data_ch.sort_values('mvel1',ascending=False).groupby('DATE').tail(1000).reset_index(drop=True)

In [42]:
# missing data before filling
data_ch.isnull().sum()

permno            0
DATE              0
mvel1           276
RET               0
prc            7830
              ...  
std_turn        410
zerotrade       342
sic2          22532
bm           375825
bm_ia        375825
Length: 101, dtype: int64

In [43]:
%%time
# fill na with cross-sectional median
for ch in characteristics:
     data_ch[ch] = data_ch.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))

CPU times: user 12.7 s, sys: 10.3 s, total: 23 s
Wall time: 24.7 s


In [44]:
# missing data after filling
data_ch.isnull().sum()

permno            0
DATE              0
mvel1             0
RET               0
prc            7830
              ...  
std_turn          0
zerotrade         0
sic2          22532
bm           104764
bm_ia        104764
Length: 101, dtype: int64

In [45]:
for ch in characteristics:
     data_ch[ch] = data_ch[ch].fillna(0)

data_ch.columns[data_ch.isnull().sum()!=0]

Index(['prc', 'mve0', 'sic2'], dtype='object')

In [46]:
def fill_na(data_ch, characteristics):
    for ch in characteristics:
         data_ch[ch] = data_ch.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))
    for ch in characteristics:
         data_ch[ch] = data_ch[ch].fillna(0)
    return data_ch

In [47]:
data_ch_top = fill_na(data_ch_top, characteristics)
data_ch_bot = fill_na(data_ch_bot, characteristics)

In [48]:
# get dummies for SIC code
def get_sic_dummies(data_ch):
    sic_dummies = pd.get_dummies(data_ch['sic2'].fillna(999).astype(int),prefix='sic').drop('sic_999',axis=1)
    data_ch_d = pd.concat([data_ch,sic_dummies],axis=1)
    data_ch_d.drop(['prc','SHROUT','mve0','sic2'],inplace=True,axis=1)
    return data_ch_d

In [49]:
data_ch_d = get_sic_dummies(data_ch)
data_ch_top_d = get_sic_dummies(data_ch_top)
data_ch_bot_d = get_sic_dummies(data_ch_bot)

In [50]:
# load macroeconomic predictors data
data_ma = pd.read_csv('/Users/jeremy_gp/Desktop/Project2/data/PredictorData/PredictorData2022.csv')
data_ma = data_ma[(data_ma['yyyymm']>=stdt//100)&(data_ma['yyyymm']<=nddt//100)].reset_index(drop=True)

# construct predictor
ma_predictors = ['dp_sp','ep_sp','bm_sp','ntis','tbl','tms','dfy','svar']
data_ma['Index'] = data_ma['Index'].str.replace(',','').astype('float64')
data_ma['dp_sp'] = data_ma['D12']/data_ma['Index']
data_ma['ep_sp'] = data_ma['E12']/data_ma['Index']
data_ma.rename({'b/m':'bm_sp'},axis=1,inplace=True)
data_ma['tms'] = data_ma['lty']-data_ma['tbl']
data_ma['dfy'] = data_ma['BAA']-data_ma['AAA']
data_ma = data_ma[['yyyymm']+ma_predictors]
data_ma['yyyymm'] = pd.to_datetime(data_ma['yyyymm'],format='%Y%m')+pd.offsets.MonthEnd(0)
data_ma.head()

,yyyymm,dp_sp,ep_sp,bm_sp,ntis,tbl,tms,dfy,svar
0,2001-01-31,0.011839,0.035490,0.150450,-0.003193,0.0515,0.0047,0.0078,0.004941
1,2001-02-28,0.012962,0.037873,0.156070,-0.006856,0.0488,0.0061,0.0077,0.002528
2,2001-03-31,0.013766,0.039161,0.133114,-0.005213,0.0442,0.0117,0.0086,0.007140
3,2001-04-30,0.012707,0.034060,0.122497,-0.002543,0.0387,0.0206,0.0087,0.007426
4,2001-05-31,0.012567,0.031592,0.120510,-0.000248,0.0362,0.0232,0.0078,0.002536


In [51]:
from sklearn.preprocessing import MinMaxScaler

def interactions(data_ch, data_ma, characteristics, ma_predictors, minmax=True):
    # construct interactions between firm characteristics and macroeconomic predictors
    data = data_ch.copy()
    data_ma_long = pd.merge(data[['DATE']],data_ma,left_on='DATE',right_on='yyyymm',how='left')
    data = data.reset_index(drop=True)
    data_ma_long = data_ma_long.reset_index(drop=True)
    for fc in characteristics:
        for mp in ma_predictors:
            data[fc+'*'+mp] = data[fc]*data_ma_long[mp]

    features = list(set(data.columns).difference({'permno','DATE','RET'})) # a list storing all 920 features used
    if minmax:
        X = MinMaxScaler((-1,1)).fit_transform(data[features])
        X = pd.DataFrame(X, columns=features)
    else:
        X = data[features]
    y = data['RET']
    print(f"The shape of the data is: {data.shape}")
    return X, y

In [52]:
# stdt_vld = np.datetime64('1975-01-31')
# stdt_tst = np.datetime64('1987-01-31')
stdt_vld = np.datetime64('2009-01-31')
stdt_tst = np.datetime64('2015-01-31')

def trn_vld_tst(data):

    # training setstdt_vld = np.datetime64('2001-01-31')
    X_trn, y_trn = interactions(data[data['DATE']<stdt_vld],data_ma[data_ma['yyyymm']<stdt_vld],characteristics,ma_predictors)

    # validation set
    X_vld, y_vld = interactions(data[(data['DATE']<stdt_tst)&(data['DATE']>=stdt_vld)],data_ma[(data_ma['yyyymm']<stdt_tst)&(data_ma['yyyymm']>=stdt_vld)],characteristics,ma_predictors)

    # testing set
    X_tst, y_tst = interactions(data[data['DATE']>=stdt_tst],data_ma[data_ma['yyyymm']>=stdt_tst],characteristics,ma_predictors)
    return X_trn, X_vld, X_tst, y_trn, y_vld, y_tst

In [53]:
%%time
X_trn, X_vld, X_tst, y_trn, y_vld, y_tst = trn_vld_tst(data_ch_top_d)

The shape of the data is: (96000, 915)
The shape of the data is: (72000, 915)
The shape of the data is: (72000, 915)
CPU times: user 2.37 s, sys: 2.27 s, total: 4.64 s
Wall time: 5.83 s


In [54]:
X_trn.head()

,chmom*dp_sp,ep*svar,cinvest*svar,grcapx*svar,saleinv*tms,sic_23,idiovol,operprof*ntis,salecash,std_turn,...,rd_mve*tbl,pchgm_pchsale*tms,herf*tms,gma*dfy,pchsale_pchrect*tms,betasq*ntis,beta*dfy,maxret*ep_sp,std_turn*dp_sp,currat*dfy
0,-0.103484,0.721112,0.593782,-0.889999,-0.824445,-1.0,-0.742554,0.532486,-0.992052,-0.897411,...,-0.920812,0.395047,-0.920474,-0.498544,0.364757,0.108676,-0.388276,-0.903560,-0.935834,-0.982410
1,-0.029352,0.721517,0.593782,-0.890094,-0.809903,-1.0,-0.854983,0.518865,-0.992289,-0.986757,...,-0.961934,0.395897,-0.871193,-0.603030,0.358424,0.101824,-0.361263,-0.895688,-0.991717,-0.984835
2,-0.031039,0.722085,0.593782,-0.888986,-0.824841,-1.0,-0.816700,0.524895,-0.885929,-0.980934,...,-1.000000,0.394993,-0.770311,-0.398834,0.351413,0.088484,-0.320317,-0.934565,-0.988075,-0.992387
3,0.030225,0.720729,0.593789,-0.884837,-0.820586,-1.0,-0.739531,0.539490,-0.991129,-0.975886,...,-0.997726,0.397029,-0.919492,-0.462679,0.357160,0.108789,-0.388783,-0.848018,-0.984917,-0.991163
4,-0.012957,0.725438,0.593773,-0.887302,-0.819556,-1.0,-0.711758,0.526025,-0.915678,-0.984042,...,-0.994849,0.395343,-0.889217,-0.521998,0.365999,0.076916,-0.291398,-0.882483,-0.990019,-0.994350


In [55]:
gc.collect()


0

In [56]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import ParameterGrid

# Loss Function
# Huber objective function
def huber(actual, predicted, xi):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    resid = actual - predicted
    huber_loss = np.where(np.abs(resid)<=xi, resid**2, 2*xi*np.abs(resid)-xi**2)
    return np.mean(huber_loss)

# Scoring Function
# out-of-sample R squared
def R_oos(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted).flatten()
    predicted = np.where(predicted<0,0,predicted)
    return 1 - (np.dot((actual-predicted),(actual-predicted)))/(np.dot(actual,actual))

# Validation Function
def val_fun(model, params: dict, X_trn, y_trn, X_vld, y_vld, illustration=True, sleep=0, is_NN=False):
    best_ros = None
    lst_params = list(ParameterGrid(params))
    for param in lst_params:
        if best_ros == None:
            if is_NN:
                mod = model().set_params(**param).fit(X_trn, y_trn, X_vld, y_vld)
            else:
                mod = model().set_params(**param).fit(X_trn, y_trn)
            best_mod = mod
            y_pred = mod.predict(X_vld)
            best_ros = R_oos(y_vld, y_pred)
            best_param = param
            if illustration:
                print(f'Model with params: {param} finished.')
                print(f'with out-of-sample R squared on validation set: {best_ros*100:.5f}%')
                print('*'*60)
        else:
            time.sleep(sleep)
            if is_NN:
                mod = model().set_params(**param).fit(X_trn, y_trn, X_vld, y_vld)
            else:
                mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            ros = R_oos(y_vld, y_pred)
            if illustration:
                print(f'Model with params: {param} finished.')
                print(f'with out-of-sample R squared on validation set: {ros*100:.5f}%')
                print('*'*60)
            if ros > best_ros:
                best_ros = ros
                best_mod = mod
                best_param = param
    if illustration:
        print('\n'+'#'*60)
        print('Tuning process finished!!!')
        print(f'The best setting is: {best_param}')
        print(f'with R2oos {best_ros*100:.2f}% on validation set.')
        print('#'*60)
    return best_mod


# Pairwise Comparison
# Diebold-Mariano test statistics

# Evaluation Output
def evaluate(actual, predicted, insample=False):
    if insample:
        print('*'*15+'In-Sample Metrics'+'*'*15)
        print(f'The in-sample R2 is {r2_score(actual,predicted)*100:.2f}%')
        print(f'The in-sample MSE is {mean_squared_error(actual,predicted):.3f}')
    else:
        print('*'*15+'Out-of-Sample Metrics'+'*'*15)
        print(f'The out-of-sample R2 is {R_oos(actual,predicted)*100:.2f}%')
        print(f'The out-of-sample MSE is {mean_squared_error(actual,predicted):.3f}')

In [57]:
%%time
time.sleep(10)
from sklearn.linear_model import LinearRegression

# OLS with all features
OLS = LinearRegression().fit(X_trn,y_trn)
evaluate(y_trn, OLS.predict(X_trn), insample=True)
evaluate(y_tst, OLS.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 19.40%
The in-sample MSE is 0.009
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -86.03%
The out-of-sample MSE is 0.054
CPU times: user 43.8 s, sys: 31.1 s, total: 1min 14s
Wall time: 22.8 s


In [58]:
%%time
from sklearn.linear_model import LinearRegression

# OLS with preselected size, bm, and momentum covariates
features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_3 = LinearRegression().fit(X_trn[features_3],y_trn)
evaluate(y_trn, OLS_3.predict(X_trn[features_3]), insample=True)
evaluate(y_tst, OLS_3.predict(X_tst[features_3]))

***************In-Sample Metrics***************
The in-sample R2 is 0.45%
The in-sample MSE is 0.011
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.45%
The out-of-sample MSE is 0.009
CPU times: user 127 ms, sys: 244 ms, total: 371 ms
Wall time: 61 ms


In [59]:
gc.collect()

0

In [60]:
%%time
time.sleep(10)
from sklearn.linear_model import HuberRegressor

# OLS by Huber robust objective function with all features
epsilon = np.max(((y_trn-OLS.predict(X_trn)).quantile(.999),1))
OLS_H = HuberRegressor(epsilon=epsilon).fit(X_trn,y_trn)
evaluate(y_trn, OLS_H.predict(X_trn), insample=True)
evaluate(y_tst, OLS_H.predict(X_tst))

***************In-Sample Metrics***************
The in-sample R2 is 13.48%
The in-sample MSE is 0.010
***************Out-of-Sample Metrics***************
The out-of-sample R2 is -3.05%
The out-of-sample MSE is 0.010
CPU times: user 2min 24s, sys: 2min 49s, total: 5min 13s
Wall time: 55.3 s


In [61]:
%%time
from sklearn.linear_model import HuberRegressor

# OLS by Huber robust objective function
# with preselected size, bm, and momentum covariates
epsilon = np.max(((y_trn-OLS_3.predict(X_trn[features_3])).quantile(.999),1))
features_3 = ['mvel1','bm','mom1m','mom6m','mom12m','mom36m']
OLS_H_3 = HuberRegressor(epsilon=epsilon).fit(X_trn[features_3],y_trn)

evaluate(y_trn, OLS_H_3.predict(X_trn[features_3]), insample=True)
evaluate(y_tst, OLS_H_3.predict(X_tst[features_3]))

***************In-Sample Metrics***************
The in-sample R2 is 0.23%
The in-sample MSE is 0.011
***************Out-of-Sample Metrics***************
The out-of-sample R2 is 0.61%
The out-of-sample MSE is 0.009
CPU times: user 487 ms, sys: 855 ms, total: 1.34 s
Wall time: 221 ms


In [62]:
gc.collect()

57

In [70]:
time.sleep(10)
from sklearn.linear_model import ElasticNet

params = {'alpha':[1e-4,.1]}
EN_sk = val_fun(ElasticNet,params=params,X_trn=X_trn,y_trn=y_trn,X_vld=X_vld,y_vld=y_vld)

# %%time
evaluate(y_trn, EN_sk.predict(X_trn), insample=True)
evaluate(y_tst, EN_sk.predict(X_tst))

Model with params: {'alpha': 0.0001} finished.
with out-of-sample R squared on validation set: -84.01906%
************************************************************
Model with params: {'alpha': 0.1} finished.
with out-of-sample R squared on validation set: 0.62828%
************************************************************

############################################################
Tuning process finished!!!
The best setting is: {'alpha': 0.1}
with R2oos 0.63% on validation set.
############################################################


UsageError: Line magic function `%%time` not found.
